In [79]:
import numpy as np 
import pandas as pd

## Punto 2:

Hallar los caudales de la tubería del sistema contra incendios de una bodega que tiene la distribución que se ve en la figura. 

Además, hallar la carga piezométrica en cada uno de los nodos, teniendo el cuenta que el tanque de abastecimiento está a un nivel de 100 m y los nodos del sistema están a una altura física de 70 m.

### Inicio de la solución

El tubo que va del tanque a la red debe proveer un caudal de 120 L/s, para que se cumpla conservación de masa en todo el sistema. Por el momento, dejaremos el tubo aparte y luego veremos qué hacer con las pérdidas, resolveremos un sistema con una entrada neta de 105 L/s en el nodo 1, hallaremos los caudales y luego, las pérdidas.

In [ ]:
# Transcripción de datos iniciales de la red cerrada (no se incluye la tubería 
# de alimentación a la misma)
Tuberia = ['12', '23', '34', '41']
Long_m = [200., 800., 200., 800.]
Diam_mm = [300., 250., 250., 250.]
CH = [95.] * 4

# Convirtiendo los datos en un dataframe para poder tener todo en un solo sitio
datos_tubos = pd.DataFrame({
    'Tuberia': Tuberia, 
    'Long (m)': Long_m,
    'Diam (mm)': Diam_mm,
    'CH': CH
}
)

# Agregando algunas columnas útiles para los cálculos posteriores
datos_tubos['Diam (m)'] = datos_tubos['Diam (mm)'] / 1000
datos_tubos['K'] = 10.674 * datos_tubos['Long (m)'] / \
(datos_tubos['CH'] * datos_tubos['Diam (m)'] ** 2.63) ** 1.85

# Imprimiendo los datos para verificación
datos_tubos

,Tuberia,Long (m),Diam (mm),CH,Diam (m),K
0,12,200.0,300.0,95.0,0.30,163.921175
1,23,800.0,250.0,95.0,0.25,1592.030573
2,34,200.0,250.0,95.0,0.25,398.007643
3,41,800.0,250.0,95.0,0.25,1592.030573


## Ecuaciones de gobierno del sistema

Se usarán 3 ecuaciones de conservación de masa y la de conservación de energía en el ecircuito. Se tienen, entonces, 4 ecuaciones para 4 incógnitas. Para los sentidos de los caudales, se asumirá que todos son *positivos*. Es decir, que van circulando en sentido horario. Las ecuaciones de conservación de masa quedan:

$$Q_{12} - Q_{41} - 0.120\ m^3/s + 0.015\ m^3/s = 0 \rightarrow Q_{12} - Q_{41} = 0.105\ m^3/s$$
$$-Q_{12} + Q_{23} = -0.03\ m^3/s$$
$$-Q_{23} + Q_{34} = -0.06\ m^3/s$$

La ecuación de conservación de energía en el circuito es: 

$$K_{12}|Q_{12}|^{0.85}+K_{23}|Q_{23}|^{0.85}+K_{34}|Q_{34}|^{0.85}+K_{41}|Q_{41}|^{0.85}=0$$

Dentro de la matriz, cada valor es un $K_{ij}|Q_{ij}|^{0.85}$. A cada uno de los coeficientes se le llamará $C_{ij}$. El cálculo de dicho $C$ se hará mediante una función y se guardará, en cada paso, en el dataframe de información sobre la tubería. No se guaradrán valores intermedios, ya que estos no son importantes para el desarrollo del ejercicio.

In [ ]:
# Definiendo una matriz cuadrada de ceros para poder poner los coeficientes
Mat_coef = np.zeros((len(datos_tubos), len(datos_tubos)))

# Orden de los tubos: 12[0] - 23[1] - 34[2] - 41[3]
# Poniendo las ecuaciones de conservación de masa (no cambian en iteraciones)
Mat_coef[0, 0] = 1
Mat_coef[0, 3] = -1

Mat_coef[1, 0] = -1
Mat_coef[1, 1] = 1

Mat_coef[2, 1] = -1
Mat_coef[2, 2] = 1

# Ahora se define una función para acutalizar los valores de la última fila de 
# la matriz
def Llenar_mat(A, Q, datos_tubos):

    datos_tubos['Q'] = Q
    datos_tubos['C'] = np.abs(datos_tubos['Q']) ** 0.85 * datos_tubos['K']

    A[3, :] = datos_tubos['C']

    return A

# Probando con valores arbitrarios
Q = [0.3, 0.5, 0.5, 0.5]
Mat_coef = Llenar_mat(Mat_coef, Q, datos_tubos)

# Definiendo el vector de mano derecha del sistema
b = np.array([0.105, -0.030, -0.06, 0.])

# Imprmiendo la prueba para poder revisar si todo está bien llenado
print('La matriz de coeficientes  con caudales de prueba queda:\n')
print(np.round(Mat_coef, 3))
print('\nEl vector de mano derecha de la ecuación:\n')
print(b)

La matriz de coeficientes  con caudales de prueba queda:

[[  1.      0.      0.     -1.   ]
 [ -1.      1.      0.      0.   ]
 [  0.     -1.      1.      0.   ]
 [ 58.91  883.234 220.809 883.234]]

El vector de mano derecha de la ecuación:

[ 0.105 -0.03  -0.06   0.   ]


In [ ]:
# Parámetros numéricos de solver
err = 1e10
tol = 1e-4
count = 1

# Suposición inicial de caudales
Q0 = np.array([0.1] * 4)

# Iterando para resolver el sistema
while err > tol: 

    # Llenar la matriz
    Mat_coef = Llenar_mat(Mat_coef, Q0, datos_tubos)

    # Resolver el sistema
    Q1 = np.linalg.solve(Mat_coef, b)
    
    # Calcular el error
    err = np.linalg.norm(Q1 - Q0)

    # Imprmir resultado parcial
    print(f'\nIteración: {count}\tError: {err:5f}')
    print('Caudales: ' + str(np.round(Q1, 4)) + '\n')
    print('Matriz de coeficientes:\n')
    print(np.round(Mat_coef, 3))
    
    # Reemplazar valores (se promedian valor anterior y posterior para reducir 
    # oscilaciones espurias en el método de solución)
    count += 1
    Q0 = 0.5 * (Q1 + Q0)


Iteración: 1	Error: 0.198182
Caudales: [ 0.0669  0.0369 -0.0231 -0.0381]

Matriz de coeficientes:

[[  1.      0.      0.     -1.   ]
 [ -1.      1.      0.      0.   ]
 [  0.     -1.      1.      0.   ]
 [ 23.154 224.88   56.22  224.88 ]]

Iteración: 2	Error: 0.121962
Caudales: [ 0.0545  0.0245 -0.0355 -0.0505]

Matriz de coeficientes:

[[  1.      0.      0.     -1.   ]
 [ -1.      1.      0.      0.   ]
 [  0.     -1.      1.      0.   ]
 [ 19.858 162.974  24.959  83.03 ]]

Iteración: 3	Error: 0.085804
Caudales: [ 0.0415  0.0115 -0.0485 -0.0635]

Matriz de coeficientes:

[[  1.      0.      0.     -1.   ]
 [ -1.      1.      0.      0.   ]
 [  0.     -1.      1.      0.   ]
 [ 16.889 117.278   1.575  31.113]]

Iteración: 4	Error: 0.015458
Caudales: [ 0.0678  0.0378 -0.0222 -0.0372]

Matriz de coeficientes:

[[ 1.     0.     0.    -1.   ]
 [-1.     1.     0.     0.   ]
 [ 0.    -1.     1.     0.   ]
 [13.984 78.529 16.415 95.749]]

Iteración: 5	Error: 0.005524
Caudales: [ 0.0657  0.

## Cálculo de pérdidas de energía

A diferencia del método de Cross, no es posible cuantificar las pérdidas a medida que se va iterando. Entonces, ya con los caudales finales se procede a calcular las pérdidas en cada uno de los tramos. Luego, Habiendo asignado la altura de cada uno de los nodos en $z = 70\ m$, se procede a calcular la carga de presión en cada uno de ellos. 

In [83]:
# Poniendo la respuesta en la columna de caudales
datos_tubos['Q'] = np.abs(Q1)
datos_tubos['Q (L/s)'] = np.round(datos_tubos['Q'] * 1000, 3)

# Asignando el sentido de giro en el dataframe
datos_tubos['Sentido'] = (Q1 / np.abs(Q1))

# Calculando la pérdida
datos_tubos['he (m)'] = np.round(datos_tubos['K'] * datos_tubos['Q'] ** 1.85 *\
datos_tubos['Sentido'], 3)

# Presentando los resultados de los caudales y las pérdidas por tramo
datos_tubos[['Tuberia', 'Q (L/s)', 'he (m)']]

,Tuberia,Q (L/s),he (m)
0,12,65.692,1.064
1,23,35.692,3.344
2,34,24.308,-0.411
3,41,39.308,-3.997


In [ ]:
# Comprobando que la suma de las pérdidas de carga con sentido es igual a 0
print('Suma de las pérdidas de carga en el circuito: ')
print(np.round(sum(datos_tubos['he (m)']), 4), ' m')

Suma de las pérdidas de carga en el circuito: 
0.0  m


In [ ]:
# Creando un dataframe de nodos para mostrar la carga en cada uno de ellos
Nodos = ['A', '1', '2', '3', '4']

datos_nodos = pd.DataFrame({
    'Nodo': Nodos
})

Carga = np.zeros(len(Nodos))

Carga[0] = 100
Carga[1] = 100 - 10.674 * 100 * (0.12 / (95 * 0.4 ** 2.63)) ** 1.85
Carga[2] = Carga[1] - datos_tubos['he (m)'][0]
Carga[3] = Carga[2] - datos_tubos['he (m)'][1]
Carga[4] = Carga[3] - datos_tubos['he (m)'][2]

datos_nodos['CargaP (m)'] = np.round(Carga, 3)

datos_nodos

,Nodo,CargaP (m)
0,A,100.000
1,1,99.600
2,2,98.536
3,3,95.192
4,4,95.603
